# source code

- single frame
- dataclass
- **image**

In [1]:
figpath = '2022-12-02_caustique_single_dataclass_image'
if figpath is None:
    import datetime
    date = datetime.datetime.now().date().isoformat()
    figpath = f'{date}_caustique'

In [2]:
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt

In [3]:
PRECISION = 4 # the higher, the bigger the file

In [4]:
PRECISION = 10 # the higher, the bigger the file

In [5]:
import os
import datetime
import numpy as np

# https://docs.python.org/3/library/dataclasses.html?highlight=dataclass#module-dataclasses
from dataclasses import dataclass, asdict, field

@dataclass
class Params:
    figpath: str = figpath # Folder to store images
    fig_width: float = 12 # width of figure
    phi: float = 1.61803 # beauty is gold
    N_PG_show: float = 5 # number of PG to show in plot_PG
    tag: str = 'caustique' # Tag
    ext: str = 'png' # Extension for output
    nx: int = 5*2**PRECISION # number of pixels (vertical)
    ny: int = 5*2**PRECISION # number of pixels (horizontal)
    nframe: int = 1 # number of frames
    bin_dens: int = 4 # relative bin density
    bin_spectrum: int = 3 # bin spacing in spectrum
    seed: int = 8601 # seed for RNG
    H: float = 1.5 # depth of the pool
    variation: float = .4 # variation of diffraction index: http://www.philiplaven.com/p20.html 1.40 at 400 nm and 1.37 at 700nm makes a 2% variation
    scale: float = 200 # sf
    B_sf: float = 0.25 # bandwidth in sf
    V_Y: float = 0.3 # horizontal speed
    V_X: float = 0.3 # vertical speed
    B_V: float = 1.0 # bandwidth in speed
    theta: float = 2*np.pi*(2-1.61803) # angle with the horizontal
    B_theta: float = 2*np.pi/3 # bandwidth in theta
    min_lum: float = .2 # diffusion level for the rendering
    gamma: float = 2.8 # Gamma exponant to convert luminosity to luminance
    fps: float = 18 # frames per second
    multispectral: bool = True # Compute caustics on the full spectrogram.
    cache: bool = True # Cache intermediate output.
    verbose: bool = False # Displays more verbose output.
    do_display: bool = False # Displays images in notebook.

In [6]:
opt = Params()
opt.figpath = figpath

In [7]:
opt

Params(figpath='2022-12-02_caustique_single_dataclass_image', fig_width=12, phi=1.61803, N_PG_show=5, tag='caustique', ext='png', nx=5120, ny=5120, nframe=1, bin_dens=4, bin_spectrum=3, seed=8601, H=1.5, variation=0.4, scale=200, B_sf=0.25, V_Y=0.3, V_X=0.3, B_V=1.0, theta=2.399988291783386, B_theta=2.0943951023931953, min_lum=0.2, gamma=2.8, fps=18, multispectral=True, cache=True, verbose=False, do_display=False)

## utilities

Transfoming a sequence of PNG frames into gif or mp4:

In [8]:
def make_gif(gifname, fnames, fps, do_delete=True):
    import imageio

    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for fname in fnames:
            writer.append_data(imageio.imread(fname))

    from pygifsicle import optimize
    optimize(str(gifname))
    if do_delete: 
        for fname in fnames: os.remove(fname)
    return gifname

# https://moviepy.readthedocs.io/en/latest/getting_started/videoclips.html#imagesequenceclip
def make_mp4(mp4name, fnames, fps, do_delete=True):
    import moviepy.editor as mpy
    clip = mpy.ImageSequenceClip(fnames, fps=fps)
    clip.write_videofile(mp4name, fps=fps, codec='libx264', verbose=False, logger=None)
    if do_delete: 
        for fname in fnames: os.remove(fname)
    return mp4name


Utilities to compute the spectrum of the blue sky and convert them later to RGB values (check out https://laurentperrinet.github.io/sciblog/posts/2020-07-04-colors-of-the-sky.html for details)

In [9]:
from lambda2color import Lambda2color, xyz_from_xy

# borrowed from https://github.com/gummiks/gummiks.github.io/blob/master/scripts/astro/planck.py
def planck(wav, T):
    import scipy.constants as const
    c = const.c # c = 3.0e+8
    h = const.h # h = 6.626e-34
    k = const.k # k = 1.38e-23
    a = 2.0*h*c**2
    b = h*c/(wav*k*T)
    intensity = a / ( (wav**5) * (np.exp(b) - 1.0) )
    return intensity

def scattering(wav, a=0.005, p=1.3, b=0.45):
    """
    b is  proportionate  to  the  column  density  of  aerosols
    along  the  path  of  sunlight,  from  outside  the  atmosphere
    to  the  point  of  observation
    
    see https://laurentperrinet.github.io/sciblog/posts/2020-07-04-colors-of-the-sky.html for more details

    """
    # converting wav in µm:
    intensity = np.exp(-a/((wav/1e-6)**4)) # Rayleigh extinction by nitrogen
    intensity *= (wav/1e-6)**-4
    intensity *= np.exp(-b/((wav/1e-6)**p)) # Aerosols
    return intensity

## computing the caustics

In [10]:
import shutil
from tqdm.notebook import tqdm, trange
import MotionClouds as mc
class Caustique:
    def __init__(self, opt):
        """
        Image coordinates follow 'ij' indexing, that is,
        * their origin at the top left,
        * the X axis is vertical and goes "down",
        * the Y axis is horizontal and goes "right".

        """
        self.mc = mc
        self.ratio = opt.ny/opt.nx # ratio between height and width (>1 for portrait, <1 for landscape)
        X = np.linspace(0, 1, opt.nx, endpoint=False) # vertical
        Y = np.linspace(0, self.ratio, opt.ny, endpoint=False) # horizontal
        self.xv, self.yv = np.meshgrid(X, Y, indexing='ij')
        self.opt = opt
        # https://stackoverflow.com/questions/16878315/what-is-the-right-way-to-treat-python-argparse-namespace-as-a-dictionary
        self.d = vars(opt)
        os.makedirs(self.opt.figpath, exist_ok=True)
        self.cachepath = os.path.join('/tmp', self.opt.figpath)
        if opt.verbose: print(f'{self.cachepath=}')
        os.makedirs(self.cachepath, exist_ok=True)

        # a standard white:
        illuminant_D65 = xyz_from_xy(0.3127, 0.3291), 
        illuminant_sun = xyz_from_xy(0.325998, 0.335354)
        # color conversion class
        self.cs_srgb = Lambda2color(red=xyz_from_xy(0.64, 0.33),
                               green=xyz_from_xy(0.30, 0.60),
                               blue=xyz_from_xy(0.15, 0.06),
                               white=illuminant_sun)
        self.wavelengths = self.cs_srgb.cmf[:, 0]*1e-9
        self.N_wavelengths = len(self.wavelengths)
        # multiply by the spectrum of the sky
        intensity5800 = planck(self.wavelengths, 5800.)
        scatter = scattering(self.wavelengths)
        self.spectrum_sky = intensity5800 * scatter
        self.spectrum_sky /= self.spectrum_sky.max()



    def wave(self):
        filename = f'{self.cachepath}/{self.opt.tag}_wave.npy'
        if os.path.isfile(filename):
            z = np.load(filename)
        else:
            # A simplistic model of a wave using https://github.com/NeuralEnsemble/MotionClouds
            fx, fy, ft = mc.get_grids(self.opt.nx, self.opt.ny, self.opt.nframe)
            env = mc.envelope_gabor(fx, fy, ft, V_X=self.opt.V_Y, V_Y=self.opt.V_X, B_V=self.opt.B_V,
                                    sf_0=1./self.opt.scale, B_sf=self.opt.B_sf/self.opt.scale,
                                    theta=self.opt.theta, B_theta=self.opt.B_theta)
            z = mc.rectif(mc.random_cloud(env, seed=self.opt.seed))
            if self.opt.cache: np.save(filename, z)
        return z

    def transform(self, z_, modulation=1.):
        xv, yv = self.xv.copy(), self.yv.copy()

        dzdx = z_ - np.roll(z_, 1, axis=0)
        dzdy = z_ - np.roll(z_, 1, axis=1)
        xv = xv + modulation * self.opt.H * dzdx
        yv = yv + modulation * self.opt.H * dzdy

        xv = np.mod(xv, 1)
        yv = np.mod(yv, self.ratio)

        return xv, yv

    def plot(self, z, image=None, do_color=True, output_filename=None, dpi=50):

        # do the raytracing of image through z:
        binsx, binsy = self.opt.nx//self.opt.bin_dens, self.opt.ny//self.opt.bin_dens

        # a fixed image in degree of contrast (from 0=black to 1=white)
        if image is None: image = np.ones((self.opt.nx, self.opt.ny))

        if output_filename is None:
            output_filename=f'{self.opt.figpath}/{self.opt.tag}.{self.opt.ext}'

        subplotpars = matplotlib.figure.SubplotParams(left=0., right=1., bottom=0., top=1., wspace=0., hspace=0.,)

        if self.opt.multispectral:

            #image_rgb = self.cs_srgb.spec_to_rgb(hist)
            image_rgb = np.zeros((self.opt.nx//self.opt.bin_dens,  self.opt.ny//self.opt.bin_dens, 3, self.opt.nframe))
            for i_frame in tqdm(range(self.opt.nframe)):
                for ii_wavelength, i_wavelength in enumerate(range(self.opt.bin_spectrum//2, self.N_wavelengths, self.opt.bin_spectrum)):
                    modulation = 1. + self.opt.variation/2 - self.opt.variation*i_wavelength/self.N_wavelengths
                    # print(i_wavelength, N_wavelengths, modulation)
                    xv, yv = self.transform(z[:, :, i_frame], modulation=modulation)
                    hist_, edge_x, edge_y = np.histogram2d(xv.ravel(), yv.ravel(),
                                                           weights=image.ravel(),
                                                           bins=[binsx, binsy],
                                                           range=[[0, 1], [0, self.ratio]],
                                                           density=True)

                    spec = np.zeros((self.N_wavelengths))
                    spec[i_wavelength] = 1
                    rgb = self.cs_srgb.spec_to_rgb(spec)
                    rgb *= self.spectrum_sky[i_wavelength]
                    image_rgb[:, :, :, i_frame] += hist_[:, :, None] * rgb[None, None, :]

            # image_rgb -= image_rgb.min()
            image_rgb /= image_rgb.max()
        else:
            hist = np.zeros((binsx, binsy, self.opt.nframe))
            for i_frame in trange(self.opt.nframe):
                xv, yv = self.transform(z[:, :, i_frame])
                hist_, edge_x, edge_y = np.histogram2d(xv.ravel(), yv.ravel(),
                                                       bins=[binsx, binsy],
                                                       range=[[0, 1], [0, self.ratio]],
                                                       density=True)
            #hist /= hist.max()

        # transform light into image:
        fnames = []
        for i_frame in range(self.opt.nframe):
            fig, ax = plt.subplots(figsize=(self.opt.ny/self.opt.bin_dens/dpi, self.opt.nx/self.opt.bin_dens/dpi), subplotpars=subplotpars)
            if self.opt.multispectral:
                ax.imshow(image_rgb[:, :, :, i_frame] ** (1/self.opt.gamma), vmin=0, vmax=1)
            else:
                if do_color:
                    bluesky = np.array([0.268375, 0.283377]) # xyz
                    sun = np.array([0.325998, 0.335354]) # xyz
                    # ax.pcolormesh(edge_y, edge_x, hist[:, :, i_frame], vmin=0, vmax=1, cmap=plt.cm.Blues_r)
                    # https://en.wikipedia.org/wiki/CIE_1931_color_space#Mixing_colors_specified_with_the_CIE_xy_chromaticity_diagram
                    L1 = 1 - hist[:, :, i_frame]
                    L2 = hist[:, :, i_frame]
                    image_denom = L1 / bluesky[1] + L2 / sun[1]
                    image_x = (L1 * bluesky[0] / bluesky[1] + L2 * sun[0] / sun[1]) / image_denom
                    image_y = (L1 + L2) / image_denom 
                    image_xyz = np.dstack((image_x, image_y, 1 - image_x - image_y))
                    image_rgb = self.cs_srgb.xyz_to_rgb(image_xyz)
                    image_L = self.opt.min_lum + (1-self.opt.min_lum)* L2 ** .61803
                    ax.imshow(image_L[:, :, None]*image_rgb, vmin=0, vmax=1)
                else:
                    ax.imshow(1-image_L, vmin=0, vmax=1)

            fname = f'{self.cachepath}/{self.opt.tag}_frame_{i_frame:04d}.png'
            fig.savefig(fname, dpi=dpi)
            fnames.append(fname)
            plt.close('all')

        if self.opt.nframe==1:
            imagefname = f'{self.opt.figpath}/{self.opt.tag}.png'
            shutil.copyfile(fname, imagefname)
            return imagefname
        else:
            if self.opt.ext == 'gif':
                return make_gif(output_filename, fnames, fps=self.opt.fps)
            else:
                return make_mp4(output_filename, fnames, fps=self.opt.fps)

    def show(self, output_filename, width=1024):
        from IPython.display import HTML, Image, display
        if self.opt.nframe==1:
            display(Image(url=output_filename.replace(self.opt.ext, 'png'), width=width))
        else:
            if self.opt.ext == 'gif':
                return display(Image(url=output_filename, width=width))
            else:
                #import moviepy.editor as mpy
                #return mpy.ipython_display(output_filename, width=width)
                # https://github.com/NeuralEnsemble/MotionClouds/blob/master/MotionClouds/MotionClouds.py#L858
                opts = ' loop="1" autoplay="1" controls '
                html = HTML(f'<video {opts} src="{output_filename}" type="video/{self.opt.ext}" width={width}\>')
                html.reload()
                return display(html)

# a simple caustics

## generating the caustics

In [11]:
c = Caustique(opt)
z = c.wave()
z.shape

(5120, 5120, 1)

In [12]:
output_filename = f'{opt.figpath}/{opt.tag}.{opt.ext}'
if not os.path.isfile(output_filename):
    c = Caustique(opt)
    z = c.wave()
    output_filename = c.plot(z)

In [13]:
if opt.do_display: c.show(output_filename)

# exploring parameters

In [14]:
N_scan = 9
base = 2

## water depth

In [15]:
opt = Params()
opt.figpath = figpath

c = Caustique(opt)
# compute just once
z = c.wave()

for H_ in c.opt.H*np.logspace(-1, 1, N_scan, base=base):
    opt = Params()
    opt.figpath = figpath
    c = Caustique(opt)

    print(f'H = {H_:.3f}')
    c.opt.H = H_
    c.opt.tag = f'{opt.tag}_H_{H_:.3f}'
    output_filename = f'{opt.figpath}/{c.opt.tag}.{opt.ext}'
    if not os.path.isfile(output_filename):
        url=c.plot(z, output_filename=output_filename)
    if opt.do_display: c.show(output_filename)

H = 0.750
H = 0.892
H = 1.061
H = 1.261
H = 1.500
H = 1.784
H = 2.121
H = 2.523
H = 3.000


## refraction index variation

In [16]:
opt = Params()
opt.figpath = figpath

c = Caustique(opt)
z = c.wave()

for variation_ in np.logspace(-2, 0, N_scan, base=10, endpoint=False):
    opt = Params()
    opt.figpath = figpath
    c = Caustique(opt)
    print(f'variation = {variation_:.3f}')
    c.opt.variation = variation_
    c.opt.tag = f'{opt.tag}_variation_{variation_:.3f}'
    output_filename = f'{opt.figpath}/{c.opt.tag}.{opt.ext}'
    if not os.path.isfile(output_filename):
        url=c.plot(z, output_filename=output_filename)
    if opt.do_display: c.show(output_filename)

variation = 0.010
variation = 0.017
variation = 0.028
variation = 0.046
variation = 0.077
variation = 0.129
variation = 0.215
variation = 0.359
variation = 0.599


## other variables

In [17]:
for variable in ['scale', 'B_sf', 'B_theta', 'V_X', 'B_V', 'gamma', ]: #  'theta', 'V_Y'
    print(f'======{variable}======')
    for modul in np.logspace(-1, 1, N_scan, base=base):
        opt = Params()
        opt.figpath = figpath

        c = Caustique(opt)
        c.d[variable] *= modul
        c.opt.tag = f'{opt.tag}_{variable}_modul_{modul:.3f}'
        output_filename = f'{opt.figpath}/{c.opt.tag}.{opt.ext}'

        print(f'{variable}={variable}(default)*{modul:.3f}={c.d[variable]:.3E}')
        if not os.path.isfile(output_filename):
            print('Doing ', output_filename)
            z = c.wave()
            mcname = f'{opt.figpath}/{c.opt.tag}'
            if False: #not os.path.isfile(f'{mcname}{c.mc.ext}'): 
                print('Doing ', f'{mcname}{c.mc.ext}')
                c.mc.anim_save(z.swapaxes(0, 1), f'{mcname}')
            url=c.plot(z, output_filename=output_filename)
        if opt.do_display: c.show(output_filename)

======scale======
scale=scale(default)*0.500=1.000E+02
scale=scale(default)*0.595=1.189E+02
scale=scale(default)*0.707=1.414E+02
scale=scale(default)*0.841=1.682E+02
scale=scale(default)*1.000=2.000E+02
scale=scale(default)*1.189=2.378E+02
scale=scale(default)*1.414=2.828E+02
scale=scale(default)*1.682=3.364E+02
scale=scale(default)*2.000=4.000E+02
======B_sf======
B_sf=B_sf(default)*0.500=1.250E-01
B_sf=B_sf(default)*0.595=1.487E-01
B_sf=B_sf(default)*0.707=1.768E-01
B_sf=B_sf(default)*0.841=2.102E-01
B_sf=B_sf(default)*1.000=2.500E-01
B_sf=B_sf(default)*1.189=2.973E-01
B_sf=B_sf(default)*1.414=3.536E-01
B_sf=B_sf(default)*1.682=4.204E-01
B_sf=B_sf(default)*2.000=5.000E-01
======B_theta======
B_theta=B_theta(default)*0.500=1.047E+00
B_theta=B_theta(default)*0.595=1.245E+00
B_theta=B_theta(default)*0.707=1.481E+00
B_theta=B_theta(default)*0.841=1.761E+00
B_theta=B_theta(default)*1.000=2.094E+00
B_theta=B_theta(default)*1.189=2.491E+00
B_theta=B_theta(default)*1.414=2.962E+00
B_theta=B_

In [18]:

c = Caustique(Params())
c.d['fps'] = 42
c.d['fps'], c.opt.fps
        

(42, 42)


## installation

Install [dependencies](https://pip.pypa.io/en/stable/user_guide/#requirements-files), then this notebook:

```
python3 -m pip install -r requirements.txt
```

In [19]:
# %pip install --upgrade -r requirements.txt